# Diffusion Transformer

## Libraries

In [1]:
import os
import gc
import math
import numpy as np
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as vutils
from torchvision import transforms
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, Dataset
from accelerate import Accelerator
from tqdm.auto import tqdm
from einops import rearrange

import matplotlib.pyplot as plt
from ema_pytorch import EMA
from PIL import Image
from torchvision.transforms import ToPILImage

## Data preprocess

In [ ]:
images = np.load('sprites.npy')
labels = np.load('sprites_labels.npy')

class_ids = np.argmax(labels, axis=1)
num_classes = labels.shape[1]
classes = ['human', 'non_human', 'food', 'spell', 'sideface']
for cls in range(num_classes):
    cls_mask = (class_ids==cls)
    class_imgs = images[cls_mask]
    out_name = f'images_class_{classes[cls]}.npy'
    np.save(out_name, class_imgs)
    print(f"Class {cls}: {class_imgs.shape[0]} images saved to {out_name}")

In [ ]:

def show_npy_image(file_path):
    """
    Load an image from a .npy file and display it.
    Supports:
      • Grayscale: shape [H, W] or [1, H, W]
      • RGB:       shape [H, W, 3] or [3, H, W]
    """
    # 1) Load
    img = np.load(file_path)[0]
    
    # 2) If it’s channel-first (C×H×W), convert to H×W×C
    if img.ndim == 3 and img.shape[0] in (1, 3):
        img = img.transpose(1, 2, 0)
    
    # 3) Plot
    plt.figure(figsize=(4, 4))
    if img.ndim == 2 or (img.ndim == 3 and img.shape[2] == 1):
        plt.imshow(img.squeeze(), cmap='gray')
    else:
        plt.imshow(img)
    plt.axis('off')
    plt.title(file_path)
    plt.show()

for cls in range(5):
    path = f'images_class_{classes[cls]}.npy'  # or prepend folder, e.g. 'data/images_class{cls}.npy'
    try:
        show_npy_image(path)
    except FileNotFoundError:
        print(f"⚠️  File not found: {path}")

In [2]:
class CustomDataset(Dataset):
    def __init__(self, images_npy: str, labels_npy: str, image_size: int):
        # load images and labels
        self.sprites = np.load(images_npy)     # shape: [N, H, W, C]
        raw_labels   = np.load(labels_npy)     # shape: [N, 5] (one-hot) or [N] (ints)

        # if one-hot, convert to class indices:
        if raw_labels.ndim == 2:
            self.labels = raw_labels.argmax(axis=1)
        else:
            self.labels = raw_labels

        self.image_size = image_size
        self.transform = transforms.Compose([
            ToPILImage(),
            transforms.RandomHorizontalFlip(0.5),
            transforms.RandomResizedCrop(image_size, scale=(0.9,1.0)),
            transforms.ColorJitter(0.1,0.1,0.1,0.05),
            transforms.ToTensor(),
            transforms.Normalize((0.5,)*3, (0.5,)*3),
        ])

    def __len__(self):
        return len(self.sprites)

    def __getitem__(self, idx):
        img_np = self.sprites[idx]                # H×W×C
        img = self.transform(img_np)              # [3, H, W]
        label = int(self.labels[idx])             # scalar in {0,…,4}
        return img, label


## Noise Generation

In [3]:
def linear_beta_schedule(timesteps):
    """
    linear schedule, proposed in original paper
    """
    scale = 1000 / timesteps
    beta_start = scale * 0.0001
    beta_end = scale * 0.02
    return torch.linspace(beta_start, beta_end, timesteps, dtype = torch.float64)

# e.g. from Nichol & Dhariwal’s “improved DDPM”:
def cosine_beta_schedule(timesteps, s=0.008):
    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * math.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return betas.clamp(max=0.999)

def extract(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))

## network modules test

In [ ]:
class TransformerBackbone(nn.Module):
    def __init__(self, in_channels=3, img_size=64, patch_size=2, stride=1, emb_dim=512, depth=6, num_heads=8, time_emb_dim=128):
        super().__init__()
        self.channels = in_channels
        self.embed = ConvPatchEmbedding(
            in_channels, emb_dim, img_size,
            patch_size=patch_size, stride=stride
        )

        self.time_embedding = SinusoidalPosEmb(time_emb_dim)
        self.time_mlp = nn.Sequential(
            nn.Linear(time_emb_dim, emb_dim),
            nn.GELU(),
            nn.Linear(emb_dim, emb_dim)
        )

        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=num_heads, dim_feedforward=1024, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=depth)
        self.output_proj = nn.Sequential(
            nn.Linear(emb_dim, patch_size * patch_size * in_channels),
        )
        self.patch_size = patch_size
        self.img_size = img_size
        

    def forward(self, x, t):
        B = x.size(0)
        patches = self.embed(x)  # [B, N, D]

        t_emb = self.time_embedding(t) # [B, time_emb_dim]
        t_emb = self.time_mlp(t_emb) # [B, D]
        t_emb = t_emb.unsqueeze(1) # [B, 1, D]

        patches = patches + t_emb # Broadcast time embedding to all patches
        
        encoded = self.encoder(patches)  # [B, N, D]
        decoded = self.output_proj(encoded)  # [B, N, patch*patch*channels]
        decoded = decoded.view(B, -1, self.channels, self.patch_size, self.patch_size)  # reshape into patches
        h = w = self.img_size // self.patch_size
        decoded = rearrange(decoded, 'b (h w) c ph pw -> b c (h ph) (w pw)', h=h, w=w)
        return decoded
    
class ConvSkipViT(nn.Module):
    def __init__(
        self,
        in_channels=3,
        img_size=64,
        patch_size=2,
        stride=1,
        emb_dim=512,
        depth=6,
        num_heads=8,
        time_emb_dim=128,
    ):
        super().__init__()
        # 1) conv to lift 3→emb_dim, preserving H×W
        self.conv_in = nn.Conv2d(in_channels, emb_dim, kernel_size=3, padding=1)
        
        # 2) the Transformer expects emb_dim channels
        self.vit = TransformerBackbone(
            in_channels=emb_dim,
            img_size=img_size,
            patch_size=patch_size,
            stride=stride,
            emb_dim=emb_dim,
            depth=depth,
            num_heads=num_heads,
            time_emb_dim=time_emb_dim,
        )
        
        # 3) conv to project emb_dim→3 and add back the original RGB
        self.conv_out = nn.Conv2d(emb_dim, in_channels, kernel_size=3, padding=1)
        
        # for diffusion wrapper
        self.channels = in_channels
        self.image_size = img_size

    def forward(self, x, t):
        """
        x: [B, 3, H, W], t: [B] timesteps
        """
        # conv skip-in
        x_emb = self.conv_in(x)                     # [B, emb_dim, H, W]
        
        # pass through your ViT
        y = self.vit(x_emb, t)                      # [B, emb_dim, H, W]
        
        # conv skip-out + residual
        out = self.conv_out(y) + x                  # [B, 3, H, W]
        return out

## Diffusion process

In [4]:
# normalization functions
def normalize_to_neg_one_to_one(img):
    return img * 2 - 1

def unnormalize_to_zero_to_one(t):
    return (t + 1) * 0.5

class GaussianDiffusion(nn.Module):
    def __init__(
        self,
        model: nn.Module,
        image_size: int, 
        timesteps: int,
        beta_schedule: str = 'linear',
        auto_normalize: bool = True,
        lambda_l1: float = 0.1,
    ):
        super().__init__()

        self.model = model
        self.channels = model.channels
        self.image_size = image_size
        self.num_timesteps = timesteps
        self.lambda_l1 = lambda_l1
        self.num_classes = model.num_classes

        # 1) build beta schedule
        if beta_schedule == 'linear':
            betas = linear_beta_schedule(timesteps)
        elif beta_schedule == 'cosine':
            betas = cosine_beta_schedule(timesteps)
        else:
            raise ValueError(f'Unknown beta schedule: {beta_schedule}')
        
        alphas = 1.0 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)

        # 2) register all buffers
        self.register_buffer('betas', betas.float())
        self.register_buffer('alphas_cumprod', alphas_cumprod.float())
        self.register_buffer('alphas_cumprod_prev', alphas_cumprod_prev.float())
        self.register_buffer('sqrt_alphas_cumprod', torch.sqrt(alphas_cumprod).float())
        self.register_buffer('sqrt_one_minus_alphas_cumprod', torch.sqrt(1 - alphas_cumprod).float())
        self.register_buffer('sqrt_recip_alphas_cumprod', torch.sqrt(1.0 / alphas_cumprod).float())
        self.register_buffer('sqrt_recipm1_alphas_cumprod', torch.sqrt(1.0 / alphas_cumprod - 1.0).float())

        # posterior q(x_{t-1} | x_t, x_0)
        posterior_variance = betas * (1 - alphas_cumprod_prev) / (1 - alphas_cumprod)
        self.register_buffer('posterior_variance', posterior_variance.float())
        self.register_buffer('posterior_log_variance_clipped', torch.log(posterior_variance.clamp(min=1e-20)).float())
        self.register_buffer('posterior_mean_coef1', (betas * torch.sqrt(alphas_cumprod_prev) / (1 - alphas_cumprod)).float())
        self.register_buffer('posterior_mean_coef2', ((1 - alphas_cumprod_prev) * torch.sqrt(alphas) / (1 - alphas_cumprod)).float())

        # 3) Normalization functions
        self.normalize = normalize_to_neg_one_to_one if auto_normalize else (lambda x: x)
        self.unnormalize = unnormalize_to_zero_to_one if auto_normalize else (lambda x: x)

    def dynamic_threshold(self, x0, percentile=0.995):
        """ Clips x0 dynamically to avoid over/under-exposed outputs """
        s = torch.quantile(x0.abs().flatten(1), percentile, dim=1)
        s = torch.maximum(s, torch.ones_like(s))[:, None, None, None]
        return x0.clamp(-s, s) / s

    def predict_start_from_noise(self, x_t, t, noise):
        """Estimate x_0 from x_t and predicted noise"""
        return (
            extract(self.sqrt_recip_alphas_cumprod, t, x_t.shape) * x_t 
            - extract(self.sqrt_recipm1_alphas_cumprod, t, x_t.shape) * noise
        )
    
    def q_posterior(self, x_start, x_t, t):
        """ Compute mean & variance of q(x_{t-1}) | x_t, x_0 """
        mean = (
            extract(self.posterior_mean_coef1, t, x_t.shape) * x_start +
            extract(self.posterior_mean_coef2, t, x_t.shape) * x_t
        )
        var = extract(self.posterior_variance, t, x_t.shape)
        log_var = extract(self.posterior_log_variance_clipped, t, x_t.shape)
        return mean, var, log_var
    
    @torch.no_grad()
    def p_sample(self, x, t: int, clip_denoised: bool = True):
        """ One diffusion reverse step """
        b = x.shape[0]
        t_batch = torch.full((b,), t, device=x.device, dtype=torch.long)
        # model predicts noise
        pred_noise = self.model(x, t_batch)
        # recover x0; optionally clamp it
        x0 = self.predict_start_from_noise(x, t_batch, pred_noise)
        x0 = self.dynamic_threshold(x0)

        # posterior mean & variance
        mean, _, log_var = self.q_posterior(x0, x, t_batch)
        noise = torch.randn_like(x) if t > 0 else 0.0
        return mean + torch.exp(0.5 * log_var) * noise

    @torch.no_grad()
    def p_sample_loop(self, shape, return_all_timesteps: bool = False):
        img = torch.randn(shape, device=self.betas.device)
        if return_all_timesteps:
            all_imgs = [img]
            for t in tqdm(reversed(range(self.num_timesteps)), desc='sampling'):
                img = self.p_sample(img, t)
                all_imgs.append(img)
            out = torch.stack(all_imgs, dim=1)
        else:
            for t in tqdm(reversed(range(self.num_timesteps)), desc='sampling'):
                img = self.p_sample(img, t)
            out = img
        return self.unnormalize(out)
    
    @torch.no_grad()
    def p_sample_cfg(self, x, t, labels, guidance_scale):
        """
        x:       [B, C, H, W]
        t:       integer timestep
        labels:  LongTensor [B] with the class IDs
        """
        b = x.shape[0]
        device = x.device

        # 1) duplicate inputs for unconditional + conditional
        x_in = torch.cat([x, x], dim=0)
        t_in = torch.full((2*b,), t, device=device, dtype=torch.long)
        # first half: uncond (we pass dummy label, e.g. zeros)
        # second half: real labels
        lbl_uncond = torch.full((b,), self.num_classes, device=device, dtype=torch.long)
        lbl_cond   = labels
        labels_in  = torch.cat([lbl_uncond, lbl_cond], dim=0)

        # 2) predict noise for both branches
        eps_all = self.model(x_in, t_in, labels_in)  # model must accept labels!
        eps_uncond, eps_cond = eps_all.chunk(2, dim=0)

        # 3) fuse via CFG
        eps = eps_uncond + guidance_scale * (eps_cond - eps_uncond)

        # 4) standard posterior
        x0 = self.predict_start_from_noise(x, t_in[:b], eps)
        x0 = self.dynamic_threshold(x0)
        mean, _, log_var = self.q_posterior(x0, x, t_in[:b])
        noise = torch.randn_like(x) if t > 0 else 0.0
        return mean + torch.exp(0.5 * log_var) * noise

    @torch.no_grad()
    def p_sample_cfg_loop(self, shape, labels, guidance_scale=0.0):
        img = torch.randn(shape, device=self.betas.device)
        for t in reversed(range(self.num_timesteps)):
            if guidance_scale > 0:
                img = self.p_sample_cfg(img, t, labels, guidance_scale)
            else:
                img = self.p_sample(img, t)
        return self.unnormalize(img)
    
    @torch.no_grad()
    def p_sample_ddim(self, x, t, t_prev, eta=0.0, clip_denoised=True):
        """ One DDIM sampling step from x_t to x_{t_prev}"""
        b = x.shape[0]
        t_batch = torch.full((b, ), t, device=x.device, dtype=torch.long)
        # predict noise
        pred_noise = self.model(x, t_batch)
        # pred x0
        x0 = self.predict_start_from_noise(x, t_batch, pred_noise)
        x0 = self.dynamic_threshold(x0)

        alpha_t = self.alphas_cumprod[t]
        alpha_prev = self.alphas_cumprod[t_prev] if t_prev >= 0 else torch.tensor(1.0, device=x.device)
        sqrt_alpha_t = alpha_t.sqrt()
        sqrt_alpha_prev = alpha_prev.sqrt()
        sigma_t = eta * ((1 - alpha_prev) / (1 - alpha_t) * (1 - alpha_t / alpha_prev)).sqrt()
        pred_dir = (1 - alpha_prev - sigma_t ** 2).sqrt() * pred_noise
        noise = sigma_t * torch.randn_like(x) if t_prev >= 0 else 0.0
        x_prev = sqrt_alpha_prev * x0 + pred_dir + noise

        return x_prev, x0
    


    @torch.no_grad()
    def ddim_sample_loop(self, shape, num_ddim_steps=50, eta=0.0, return_all_timesteps=False):
        """ Run the full DDIM sampling loop """
        device = self.betas.device
        img = torch.randn(shape, device=device)
        all_imgs = [img]

        # Create a custom timestep schedule
        ddim_timesteps = np.linspace(0, self.num_timesteps - 1, num_ddim_steps, dtype=int)

        for i in tqdm(range(num_ddim_steps - 1, -1, -1), desc='DDIM sampling'):
            t = ddim_timesteps[i]
            t_prev = ddim_timesteps[i - 1] if i > 0 else -1

            img, _ = self.p_sample_ddim(img, t, t_prev, eta=eta)
            all_imgs.append(img)
        
        out = torch.stack(all_imgs, dim=1) if return_all_timesteps else img
        return self.unnormalize(out)

    def sample(self, batch_size=16, use_ddim=False, use_cfg=False, num_ddim_steps=50, eta=0.0, return_all_timesteps=False, labels=None, guidance_scale=0.0):
        shape = (batch_size, self.channels, self.image_size, self.image_size)
        if use_ddim:
            return self.ddim_sample_loop(shape, num_ddim_steps=num_ddim_steps, eta=eta, return_all_timesteps=return_all_timesteps)
        elif use_cfg:
            if labels is None:
                labels = torch.zeros(batch_size, dtype=torch.long, device=self.betas.device)
            return self.p_sample_cfg_loop(shape, labels, guidance_scale)
        else:
            return self.p_sample_loop(shape, return_all_timesteps=return_all_timesteps)
    
    def q_sample(self, x_start, t, noise=None):
        """ Forward noising process """
        if noise is None:
            noise = torch.randn_like(x_start)
        return (
            extract(self.sqrt_alphas_cumprod, t, x_start.shape) * x_start + 
            extract(self.sqrt_one_minus_alphas_cumprod, t, x_start.shape) * noise
        )
    
    def p_losses(self, x_start, t, noise=None):
        """ MSE + L1 loss between true noise and model's prediction """
        if noise is None:
            noise = torch.randn_like(x_start)
        x_noisy = self.q_sample(x_start, t, noise)
        pred_noise = self.model(x_noisy, t)
        mse_loss = F.mse_loss(pred_noise, noise, reduction='none')
        mse_loss = mse_loss.mean(dim=list(range(1, mse_loss.ndim))) # mean over c, h, w

        x_recon = self.predict_start_from_noise(x_noisy, t, pred_noise)
        x_recon = x_recon.clamp(-1., 1.)
        l1_loss = F.l1_loss(x_recon, x_start, reduction='none')
        l1_loss = l1_loss.mean(dim=list(range(1, l1_loss.ndim)))

        loss = mse_loss + self.lambda_l1 * l1_loss
        # loss = mse_loss
        return loss.mean()
    
    def forward(self, img, labels=None):
        """ Training entrypoint: sample random timesteps & return loss """
        b, c, h, w = img.shape
        assert h == self.image_size and w == self.image_size
        t = torch.randint(0, self.num_timesteps, (b, ), device=img.device)
        img = self.normalize(img)
        return self.p_losses(img, t)

## Trainer

In [5]:
def num_to_groups(num, divisor):
    groups = num // divisor
    remainder = num % divisor
    arr = [divisor] * groups
    if remainder > 0:
        arr.append(remainder)
    return arr

def cycle(dl):
    while True:
        for data in dl:
            yield data

class Trainer:
    def __init__(
        self,
        diffusion_model: nn.Module,
        data_folder: str,
        label_folder: str,
        batch_size: int = 16,
        lr: float = 1e-4,
        num_steps: int = 100000,
        grad_accum_steps: int = 1,
        ema_decay: float = 0.995,
        save_interval: int = 1000,
        num_samples: int = 25,
        results_folder: str = './results',
        use_ddim = False,
        num_ddim_steps=50,
        eta = 0.0, 
        use_cfg = False,
        guidance_scale=2.0
    ):
        # Accelerator
        self.accelerator = Accelerator(mixed_precision='fp16')
        self.device = self.accelerator.device

        # Training State
        self.batch_size       = batch_size
        self.grad_accum_steps = grad_accum_steps
        self.num_steps        = num_steps
        self.save_interval    = save_interval
        self.num_samples      = num_samples

        # model, optimizer, EMA
        self.model = diffusion_model.to(self.device)
        self.optimizer = Adam(self.model.parameters(), lr=lr)
        self.scheduler = CosineAnnealingLR(self.optimizer, T_max=self.num_steps, eta_min=1e-6)
        self.model, self.optimizer = self.accelerator.prepare(
            self.model, self.optimizer
        )
        
        # Use EMA on the raw model
        self.ema = EMA(self.accelerator.unwrap_model(self.model), beta=ema_decay)

        # Data
        ds = CustomDataset(data_folder, label_folder, diffusion_model.image_size)
        dl = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count())
        self.dl = cycle(self.accelerator.prepare(dl))

        # checkpoints & samples
        self.results_folder = Path(results_folder)
        self.results_folder.mkdir(parents=True, exist_ok=True)
        self.step = 0
        self.use_ddim = use_ddim
        self.num_ddim_steps = num_ddim_steps
        self.eta = eta
        self.use_cfg = use_cfg
        self.guidance_scale = guidance_scale
    
    def save(self, milestone: int):
        """Save model, optimizer, EMA and step counter."""
        if not self.accelerator.is_main_process:
            return
        ckpt = self.results_folder / f'model-{milestone}.pt'
        data = {
            'step': self.step,
            'model': self.accelerator.get_state_dict(self.model),
            'optimizer': self.optimizer.state_dict(),
            'ema':       self.ema.state_dict(),
        }
        torch.save(data, ckpt)

    def load(self, ckpt_path: str):
        """Load all state (model, optimizer, EMA, step)."""
        data = torch.load(ckpt_path, map_location=self.device)
        raw_model = self.accelerator.unwrap_model(self.model)
        raw_model.load_state_dict(data['model'])
        self.optimizer.load_state_dict(data['optimizer'])
        self.ema.load_state_dict(data['ema'])
        self.step = data['step']

    def _sample_and_save(self, milestone: int, guidance_scale: float = 2.0, use_cfg=False):
        """Generate `num_samples` via EMA model and save grid."""
        self.ema.ema_model.eval()
        batches = num_to_groups(self.num_samples, self.batch_size)
        if use_cfg:
            cls = milestone % 5
            labels = torch.full((self.num_samples,), cls, dtype=torch.long, device=self.device)
            imgs = []
            offset = 0
            for n in batches:
                imgs.append(
                    self.ema.ema_model.sample(
                        batch_size=n,
                        use_cfg=True,
                        labels=labels[offset:offset+n],
                        guidance_scale=guidance_scale,
                        use_ddim=self.use_ddim,
                        num_ddim_steps=self.num_ddim_steps,
                        eta=self.eta
                    )
                )
                offset += n
            imgs = torch.cat(imgs, dim=0)
            path = self.results_folder / f'sample-{milestone}-cls{cls}-gs{guidance_scale}.png'
        else:
            imgs = torch.cat([
                self.ema.ema_model.sample(batch_size=n, use_ddim=self.use_ddim, num_ddim_steps=self.num_ddim_steps, eta=self.eta) for n in batches
            ], dim=0)
            path = self.results_folder / f'sample-{milestone}.png'
        vutils.save_image(imgs, path, nrow=int(math.sqrt(self.num_samples)))

    def train(self):
        """Run the training loop with gradient accumulation, EMA updates, and periodic sampling."""
        pbar = tqdm(total=self.num_steps, initial=self.step, disable=True)
        while self.step < self.num_steps:
            total_loss = 0.0

            # gradient accumulation
            for _ in range(self.grad_accum_steps):
                imgs, labels = next(self.dl)
                imgs, labels = imgs.to(self.device), labels.to(self.device)
                with self.accelerator.autocast():
                    loss = self.model(imgs, labels) / self.grad_accum_steps
                total_loss += loss.item()
                self.accelerator.backward(loss)

            # optimizer step
            self.accelerator.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            self.scheduler.step()
            self.optimizer.zero_grad()
            self.step += 1

            # EMA & sampling
            if self.accelerator.is_main_process:
                self.ema.update()
                if self.step % self.save_interval == 0:
                    milestone = self.step // self.save_interval
                    self._sample_and_save(milestone, self.guidance_scale, self.use_cfg)
                    self.save(milestone)
            if self.step % 10 == 0 and self.accelerator.is_main_process:
                print(f"[Step {self.step}] loss: {total_loss:.4f}")

            pbar.set_description(f'loss: {total_loss:.4f}')
            pbar.update(1)

        if self.accelerator.is_main_process:
            print('Training complete.')

    def inference(self, total: int = 1000, target_class: int=0, guidance_scale: float = 2.0, output_path: str = './submission'):
        """
        Generate `total` images using the same batch size as training
        (self.batch_size, e.g. 16), and save them to disk.
        """
        Path(output_path).mkdir(parents=True, exist_ok=True)
        count = 0
        batch_size = self.batch_size  # manually fixed at 16

        with torch.no_grad():
            while count < total:
                n = min(batch_size, total - count)

                # clear any leftover GPU memory
                if torch.cuda.is_available():
                    gc.collect()
                    torch.cuda.empty_cache()

                # sample n images
                imgs = self.ema.ema_model.sample(batch_size=n, use_ddim=self.use_ddim, num_ddim_steps=self.num_ddim_steps, eta=self.eta)

                # move to CPU and save
                imgs = imgs.cpu()
                for img in imgs:
                    count += 1
                    vutils.save_image(img, f"{output_path}/{count}.jpg")

                # clean up 
                del imgs
                if torch.cuda.is_available():
                    gc.collect()
                    torch.cuda.empty_cache()

        print("Inference complete.")

## Network modules 

## UnetTransformer

In [6]:
class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, t):
        device = t.device
        half_dim = self.dim // 2
        emb = torch.exp(
            torch.arange(half_dim, device=device) * -(math.log(10000) / (half_dim - 1))
        )
        emb = t[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb  # [B, dim]

class TransformerBlock(nn.Module):
    def __init__(
        self,
        in_channels: int,
        emb_dim: int,
        num_heads: int,
        dim_feedforward: int | None = None,
        dropout: float = 0.0
    ):
        super().__init__()
        self.input_proj = (
            nn.Conv2d(in_channels, emb_dim, kernel_size=1)
            if in_channels != emb_dim else nn.Identity()
        )

        self.self_attn = nn.MultiheadAttention(
            embed_dim=emb_dim, num_heads=num_heads,
            dropout=dropout, batch_first=True
        )

        self.cross_attn = nn.MultiheadAttention(
            embed_dim=emb_dim, num_heads=num_heads,
            dropout=dropout, batch_first=True
        )

        self.norm1 = nn.LayerNorm(emb_dim)
        self.norm2 = nn.LayerNorm(emb_dim)
        self.norm3 = nn.LayerNorm(emb_dim)

        ff_dim = dim_feedforward or (emb_dim * 4)
        self.ff = nn.Sequential(
            nn.Linear(emb_dim, ff_dim),
            nn.GELU(),
            nn.Linear(ff_dim, emb_dim),
        )

        self.output_proj = (
            nn.Conv2d(emb_dim, in_channels, kernel_size=1)
            if in_channels != emb_dim else nn.Identity()
        )

    def forward(self, x: torch.Tensor, t_emb: torch.Tensor, label_emb: torch.Tensor | None = None):
        B, C, H, W = x.shape
        x0 = self.input_proj(x)                   # [B, emb_dim, H, W]
        t = t_emb[:, :, None, None]
        x0 = x0 + t

        x_flat = rearrange(x0, 'b c h w -> b (h w) c')  # [B, N, C]
        x1, _ = self.self_attn(x_flat, x_flat, x_flat, need_weights=False)
        x1 = self.norm1(x_flat + x1)

        if label_emb is not None:
            label_emb = label_emb.unsqueeze(1)         # [B, 1, emb_dim]
            x2, _ = self.cross_attn(x1, label_emb, label_emb, need_weights=False)
            x2 = self.norm2(x1 + x2)
        else:
            x2 = x1

        ff_out = self.ff(x2)
        x3 = self.norm3(x2 + ff_out)

        x3 = rearrange(x3, 'b (h w) c -> b c h w', h=H, w=W)
        return self.output_proj(x3)

class CBAM(nn.Module): # Convolutional Block attention module
    def __init__(self, channels, reduction=16, kernel_size=7):
        super().__init__()
        self.channel_attn = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, 1, bias=False),
            nn.Sigmoid()
        )
        self.spatial_attn = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        ca = self.channel_attn(x)
        x = x * ca

        avg_pool = torch.mean(x, dim=1, keepdim=True)
        max_pool, _ = torch.max(x, dim=1, keepdim=True)
        sa = self.spatial_attn(torch.cat([avg_pool, max_pool], dim=1))
        return x * sa

class DoubleConv(nn.Module):
    """Two successive 3×3 convs each followed by GELU."""
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.GELU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.GELU(),
        )
        self.cbam = CBAM(out_ch)
    def forward(self, x):
        x = self.conv(x)
        return self.cbam(x)


class UNetTransformer(nn.Module):
    def __init__(
        self,
        *,
        in_channels=3,
        base_channels=64,
        emb_dim=256,
        num_heads=8,
        time_emb_dim=128,
        num_classes:int = 5
    ):
        super().__init__()
        self.channels = in_channels
        self.num_classes = num_classes
        # Time‐MLP
        self.time_mlp = nn.Sequential(
            SinusoidalPosEmb(time_emb_dim),
            nn.Linear(time_emb_dim, emb_dim),
            nn.GELU(),
            nn.Linear(emb_dim, emb_dim),
        )
        self.label_emb = nn.Embedding(num_classes + 1, emb_dim)

        # ---- Encoder ----
        # Stage 1: 3 → 64
        self.enc1    = DoubleConv(in_channels,        base_channels)
        self.trans1  = TransformerBlock(base_channels, emb_dim, num_heads)

        # Stage 2: 64 → 128
        self.enc2    = DoubleConv(base_channels,      base_channels*2)
        self.down2   = nn.Conv2d(base_channels*2,     base_channels*2, 4, stride=2, padding=1)
        self.trans2  = TransformerBlock(base_channels*2, emb_dim, num_heads)

        # Stage 3: 128 → 256
        self.enc3    = DoubleConv(base_channels*2,    base_channels*4)
        self.down3   = nn.Conv2d(base_channels*4,     base_channels*4, 4, stride=2, padding=1)
        self.trans3  = TransformerBlock(base_channels*4, emb_dim, num_heads)

        # Stage 4: 256 → 512
        self.enc4    = DoubleConv(base_channels*4,    base_channels*8)
        self.down4   = nn.Conv2d(base_channels*8,     base_channels*8, 4, stride=2, padding=1)
        self.trans4  = TransformerBlock(base_channels*8, emb_dim, num_heads)

        # ---- Decoder ----
        # Upsample + concat + DoubleConv + Transformer per stage
        # Stage 4→3: 512 → 256, then cat with enc4(256)→512→256
        self.up4       = nn.ConvTranspose2d(base_channels*8, base_channels*4, 4, stride=2, padding=1)
        self.dec4      = DoubleConv(base_channels*4 + base_channels*8, base_channels*4)
        self.trans_up3 = TransformerBlock(base_channels*4, emb_dim, num_heads)

        # Stage 3→2: 256 → 128, then cat with enc3(128)→256→128
        self.up3       = nn.ConvTranspose2d(base_channels*4, base_channels*2, 4, stride=2, padding=1)
        self.dec3      = DoubleConv(base_channels*2 + base_channels*4, base_channels*2)
        self.trans_up2 = TransformerBlock(base_channels*2, emb_dim, num_heads)

        # Stage 2→1: 128 → 64, then cat with enc2(64)→128→64
        self.up2       = nn.ConvTranspose2d(base_channels*2, base_channels, 4, stride=2, padding=1)
        self.dec2      = DoubleConv(base_channels + base_channels*2, base_channels)
        self.trans_up1 = TransformerBlock(base_channels, emb_dim, num_heads)

        # Final 1×1 conv to RGB
        self.out_conv = nn.Conv2d(base_channels, in_channels, 1)

    def forward(self, x, t, labels=None):
        B = x.size(0)
        # build time embedding once
        t_emb = self.time_mlp(t)  # [B, emb_dim]

        if labels is None:
            labels = torch.full((B,), self.label_emb.num_embeddings-1, device=x.device, dtype=torch.long)
        label_emb = self.label_emb(labels)

        # —— Encode ——
        e1 = self.enc1(x)               # [B,  64, 16,16]
        e1 = self.trans1(e1, t_emb, label_emb)

        x2 = self.enc2(e1)              # [B, 128,16,16]
        d2 = self.down2(x2)             # [B, 128, 8, 8]
        e2 = self.trans2(d2, t_emb, label_emb)

        x3 = self.enc3(e2)              # [B, 256, 8, 8]
        d3 = self.down3(x3)             # [B, 256, 4, 4]
        e3 = self.trans3(d3, t_emb, label_emb)

        x4 = self.enc4(e3)              # [B, 512, 4, 4]
        b  = self.down4(x4)             # [B, 512, 2, 2]
        b  = self.trans4(b, t_emb, label_emb)

        # —— Decode ——
        u3 = self.up4(b)                           # [B, 256, 4, 4]
        u3 = torch.cat([u3, x4], dim=1)            # [B, 256+512=768, 4,4]
        u3 = self.dec4(u3)                         # [B, 256, 4, 4]
        u3 = self.trans_up3(u3, t_emb, label_emb)

        u2 = self.up3(u3)                          # [B, 128, 8, 8]
        u2 = torch.cat([u2, x3], dim=1)            # [B, 128+256=384, 8,8]
        u2 = self.dec3(u2)                         # [B, 128, 8, 8]
        u2 = self.trans_up2(u2, t_emb, label_emb)

        u1 = self.up2(u2)                          # [B, 64,16,16]
        u1 = torch.cat([u1, x2], dim=1)            # [B, 64+128=192,16,16]
        u1 = self.dec2(u1)                         # [B, 64,16,16]
        u1 = self.trans_up1(u1, t_emb, label_emb)

        return self.out_conv(u1)                   # [B,  3,16,16]
    
model = UNetTransformer(
    in_channels=3,
    base_channels=64,
    emb_dim=256,
    num_heads=4,
    time_emb_dim=128,
    num_classes=5
)

## UVitPixelArt

In [ ]:
class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, t):
        device = t.device
        half_dim = self.dim // 2
        emb = torch.exp(
            torch.arange(half_dim, device=device) * -(math.log(10000) / (half_dim - 1))
        )
        emb = t[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb  # [B, dim]

class TransformerBlock(nn.Module):
    def __init__(self, dim, heads=8):
        super().__init__()
        self.attn = nn.MultiheadAttention(dim, heads, batch_first=True)
        self.norm1 = nn.LayerNorm(dim)
        self.ff = nn.Sequential(nn.Linear(dim, dim*4), nn.GELU(), nn.Linear(dim*4, dim))
        self.norm2 = nn.LayerNorm(dim)
    def forward(self, x, t_emb=None):
        if t_emb is not None:
            x = x + t_emb.unsqueeze(1)
        a, _ = self.attn(x, x, x)
        x = self.norm1(x + a)
        x = self.norm2(x + self.ff(x))
        return x

class UVitPixelArt(nn.Module):
    def __init__(
        self,
        in_channels=3,
        base_channels=64,
        emb_dim=256,
        enc8_blocks=2,
        bottleneck_blocks=4,
        dec8_blocks=2,
        dec16_blocks=1,
        heads=8,
        time_dim=128,
        num_classes = None
    ):
        super().__init__()
        self.channels = in_channels
        self.num_classes = num_classes
        if num_classes is not None:
            self.label_emb = nn.Embedding(num_classes+1, emb_dim)
        else:
            self.label_emb=None
        # time embed
        self.time_mlp = nn.Sequential(
            SinusoidalPosEmb(time_dim), nn.Linear(time_dim, emb_dim), nn.GELU(), nn.Linear(emb_dim, emb_dim)
        )
        # stem 16×16
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels, base_channels, 3, padding=1), nn.GELU(),
            nn.Conv2d(base_channels, base_channels, 3, padding=1), nn.GELU()
        )
        # encoder 16→8
        self.down8 = nn.Conv2d(base_channels, base_channels*2, 2, 2)
        self.patch8_enc = nn.Conv2d(base_channels*2, emb_dim, 3, 2, 1)
                # positional embeddings for 4×4 tokens
        self.pos8_enc = nn.Parameter(torch.randn(1, 4*4, emb_dim))
        # positional embeddings for 8×8 tokens in decoder
        self.pos8_dec = nn.Parameter(torch.randn(1, 8*8, emb_dim))
        self.enc8 = nn.ModuleList([TransformerBlock(emb_dim, heads) for _ in range(enc8_blocks)])
        # encoder 8→4
        self.down4 = nn.Conv2d(base_channels*2, base_channels*4, 2, 2)
        self.patch4 = nn.Conv2d(base_channels*4, emb_dim, 3, 1, 1)
        self.pos4 = nn.Parameter(torch.randn(1, 4*4, emb_dim))
        self.bottleneck = nn.ModuleList([TransformerBlock(emb_dim, heads) for _ in range(bottleneck_blocks)])
        # decoder 4→8
        self.up4 = nn.ConvTranspose2d(emb_dim, base_channels*4, 2, 2)
        self.skip8_proj = nn.Conv2d(base_channels*2, base_channels*4, 1)
        self.patch8_dec = nn.Conv2d(base_channels*4, emb_dim, 3, 1, 1)
        self.dec8 = nn.ModuleList([TransformerBlock(emb_dim, heads) for _ in range(dec8_blocks)])
        # decoder 8→16
        self.up8 = nn.ConvTranspose2d(emb_dim, base_channels*2, 2, 2)
        self.skip16_proj = nn.Conv2d(base_channels, base_channels*2, 1)
        self.patch16 = nn.Conv2d(base_channels*2, emb_dim, 3, 1, 1)
        self.pos16 = nn.Parameter(torch.randn(1, 16*16, emb_dim))
        self.dec16 = nn.ModuleList([TransformerBlock(emb_dim, heads) for _ in range(dec16_blocks)])
        # final conv
        self.final_conv = nn.Sequential(
            nn.Conv2d(emb_dim, base_channels, 3, padding=1), nn.GELU(),
            nn.Conv2d(base_channels, in_channels, 3, padding=1)
        )

    def forward(self, x, t, labels=None):
        B = x.size(0)
        t_emb = self.time_mlp(t)
        if self.label_emb is not None:
            if labels is None:
                labels = torch.full((x.size(0),), self.num_classes, dtype=torch.long, device=x.device)
            label_emb = self.label_emb(labels)
            t_emb = t_emb + label_emb
        # 16×16 stem
        x16 = self.stem(x)
        # 16→8 encode
        f8 = self.down8(x16)
        tok8 = self.patch8_enc(f8)              # [B,emb,4,4]
        tok8 = rearrange(tok8, 'b c h w -> b (h w) c') + self.pos8_enc + t_emb.unsqueeze(1)
        for blk in self.enc8:
            tok8 = blk(tok8, t_emb)
        # 8→4 encode
        f4 = self.down4(f8)
        tok4 = self.patch4(f4)                  # [B,emb,4,4]
        tok4 = rearrange(tok4, 'b c h w -> b (h w) c') + self.pos4 + t_emb.unsqueeze(1)
        for blk in self.bottleneck:
            tok4 = blk(tok4, t_emb)
        # 4→8 decode
        dec4 = rearrange(tok4, 'b (h w) c -> b c h w', h=4, w=4)
        up8 = self.up4(dec4)                    # [B,4C,8,8]
        skip8 = self.skip8_proj(f8)             # [B,4C,8,8]
        x8 = up8 + skip8                        # [B,4C,8,8]
        tok8_d = self.patch8_dec(x8)            # [B,emb,8,8]
        tok8_d = rearrange(tok8_d, 'b c h w -> b (h w) c') + self.pos8_dec + t_emb.unsqueeze(1)
        for blk in self.dec8:
            tok8_d = blk(tok8_d, t_emb)
        # 8→16 decode
        dec8 = rearrange(tok8_d, 'b (h w) c -> b c h w', h=8, w=8)
        up16 = self.up8(dec8)                   # [B,2C,16,16]
        skip16 = self.skip16_proj(x16)          # [B,2C,16,16]
        x16d = up16 + skip16                    # [B,2C,16,16]
        tok16 = self.patch16(x16d)              # [B,emb,16,16]
        tok16 = rearrange(tok16, 'b c h w -> b (h w) c') + self.pos16 + t_emb.unsqueeze(1)
        for blk in self.dec16:
            tok16 = blk(tok16, t_emb)
        # reconstruct and final conv
        dec16 = rearrange(tok16, 'b (h w) c -> b c h w', h=16, w=16)
        return self.final_conv(dec16)

model = UVitPixelArt(
    in_channels=3,
    base_channels=64,
    emb_dim=256,
    enc8_blocks=2,
    bottleneck_blocks=4,
    dec8_blocks=2,
    dec16_blocks=1,
    heads=8,
    time_dim=128,
    num_classes=5
)

## Usage

In [7]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# model = TransformerBackbone(
#     in_channels=3, img_size=16, patch_size=1,
#     emb_dim=1024, depth=8, num_heads=8, time_emb_dim=128
# )


In [8]:
diffusion = GaussianDiffusion(
    model=model, image_size=16, timesteps=1000,
    beta_schedule='cosine', auto_normalize=True,
    lambda_l1=0.0
)

trainer = Trainer(
    diffusion_model=diffusion,
    data_folder = './sprites.npy',
    label_folder = './sprites_labels.npy',
    batch_size=128, lr=1e-4, num_steps=100000,
    grad_accum_steps=1, ema_decay=0.999,
    save_interval=1000, num_samples=25,
    results_folder='./results',
    use_ddim=True, num_ddim_steps=100, eta=0.0,
    use_cfg=True, guidance_scale=2.0
)

In [9]:
total_params = sum(p.numel() for p in model.parameters())
print("Params: ", total_params/1e6, "M")

Params:  24.032241 M


In [10]:
trainer.train()

[Step 10] loss: 1.0097
[Step 20] loss: 1.0041
[Step 30] loss: 1.0029
[Step 40] loss: 1.0054
[Step 50] loss: 0.9968
[Step 60] loss: 0.9021
[Step 70] loss: 0.8073
[Step 80] loss: 0.7477
[Step 90] loss: 0.6716
[Step 100] loss: 0.6232
[Step 110] loss: 0.6287
[Step 120] loss: 0.5492
[Step 130] loss: 0.5477
[Step 140] loss: 0.5221
[Step 150] loss: 0.5076
[Step 160] loss: 0.5034
[Step 170] loss: 0.4463
[Step 180] loss: 0.4113
[Step 190] loss: 0.3837
[Step 200] loss: 0.4074
[Step 210] loss: 0.3843
[Step 220] loss: 0.3535
[Step 230] loss: 0.3606
[Step 240] loss: 0.3433
[Step 250] loss: 0.3507
[Step 260] loss: 0.3040
[Step 270] loss: 0.3516
[Step 280] loss: 0.3101
[Step 290] loss: 0.3406
[Step 300] loss: 0.3709
[Step 310] loss: 0.2901
[Step 320] loss: 0.3505
[Step 330] loss: 0.2807
[Step 340] loss: 0.2986
[Step 350] loss: 0.3130
[Step 360] loss: 0.2920
[Step 370] loss: 0.2945
[Step 380] loss: 0.2776
[Step 390] loss: 0.2877
[Step 400] loss: 0.2774
[Step 410] loss: 0.3103
[Step 420] loss: 0.3085
[

DDIM sampling:   0%|          | 0/100 [00:00<?, ?it/s]

[Step 1000] loss: 0.2208
[Step 1010] loss: 0.2240
[Step 1020] loss: 0.2216
[Step 1030] loss: 0.2264
[Step 1040] loss: 0.2523
[Step 1050] loss: 0.2191
[Step 1060] loss: 0.2176
[Step 1070] loss: 0.1981
[Step 1080] loss: 0.2202
[Step 1090] loss: 0.2301
[Step 1100] loss: 0.2178
[Step 1110] loss: 0.2010
[Step 1120] loss: 0.2157
[Step 1130] loss: 0.2260
[Step 1140] loss: 0.2085
[Step 1150] loss: 0.2116
[Step 1160] loss: 0.2204
[Step 1170] loss: 0.1828
[Step 1180] loss: 0.2092
[Step 1190] loss: 0.1997
[Step 1200] loss: 0.1879
[Step 1210] loss: 0.2156
[Step 1220] loss: 0.1915
[Step 1230] loss: 0.2224
[Step 1240] loss: 0.2352
[Step 1250] loss: 0.1948
[Step 1260] loss: 0.2276
[Step 1270] loss: 0.2099
[Step 1280] loss: 0.1897
[Step 1290] loss: 0.2093
[Step 1300] loss: 0.2180
[Step 1310] loss: 0.2215
[Step 1320] loss: 0.2171
[Step 1330] loss: 0.1858
[Step 1340] loss: 0.2262
[Step 1350] loss: 0.1957
[Step 1360] loss: 0.2262
[Step 1370] loss: 0.2250
[Step 1380] loss: 0.2032
[Step 1390] loss: 0.2021


DDIM sampling:   0%|          | 0/100 [00:00<?, ?it/s]

[Step 2000] loss: 0.1974
[Step 2010] loss: 0.1715
[Step 2020] loss: 0.1920
[Step 2030] loss: 0.1834
[Step 2040] loss: 0.1695
[Step 2050] loss: 0.2006
[Step 2060] loss: 0.1806
[Step 2070] loss: 0.2077
[Step 2080] loss: 0.1793
[Step 2090] loss: 0.1847
[Step 2100] loss: 0.1689
[Step 2110] loss: 0.1784
[Step 2120] loss: 0.1939
[Step 2130] loss: 0.1887
[Step 2140] loss: 0.1763
[Step 2150] loss: 0.2099
[Step 2160] loss: 0.1795
[Step 2170] loss: 0.1978
[Step 2180] loss: 0.1659
[Step 2190] loss: 0.1669
[Step 2200] loss: 0.1971
[Step 2210] loss: 0.1812
[Step 2220] loss: 0.1955
[Step 2230] loss: 0.2075
[Step 2240] loss: 0.1921
[Step 2250] loss: 0.1846
[Step 2260] loss: 0.1778
[Step 2270] loss: 0.1903
[Step 2280] loss: 0.1695
[Step 2290] loss: 0.1877
[Step 2300] loss: 0.2047
[Step 2310] loss: 0.1781
[Step 2320] loss: 0.1739
[Step 2330] loss: 0.1917
[Step 2340] loss: 0.1719
[Step 2350] loss: 0.1897
[Step 2360] loss: 0.1682
[Step 2370] loss: 0.1800
[Step 2380] loss: 0.1944
[Step 2390] loss: 0.1628


...

DDIM sampling:   0%|          | 0/100 [00:00<?, ?it/s]

[Step 34000] loss: 0.0640
[Step 34010] loss: 0.0613
[Step 34020] loss: 0.0667
[Step 34030] loss: 0.0544
[Step 34040] loss: 0.0646
[Step 34050] loss: 0.0721
[Step 34060] loss: 0.0607
[Step 34070] loss: 0.0697
[Step 34080] loss: 0.0644
[Step 34090] loss: 0.0657
[Step 34100] loss: 0.0705
[Step 34110] loss: 0.0546
[Step 34120] loss: 0.0835
[Step 34130] loss: 0.0585
[Step 34140] loss: 0.0698
[Step 34150] loss: 0.0806
[Step 34160] loss: 0.0717
[Step 34170] loss: 0.0779
[Step 34180] loss: 0.0700
[Step 34190] loss: 0.0518
[Step 34200] loss: 0.0571
[Step 34210] loss: 0.0775
[Step 34220] loss: 0.0713
[Step 34230] loss: 0.0707
[Step 34240] loss: 0.0697
[Step 34250] loss: 0.0790
[Step 34260] loss: 0.0656
[Step 34270] loss: 0.0657
[Step 34280] loss: 0.0685
[Step 34290] loss: 0.0513
[Step 34300] loss: 0.0753
[Step 34310] loss: 0.0585
[Step 34320] loss: 0.0557
[Step 34330] loss: 0.0736
[Step 34340] loss: 0.0601
[Step 34350] loss: 0.0653
[Step 34360] loss: 0.0727
[Step 34370] loss: 0.0720
[Step 34380]

DDIM sampling:   0%|          | 0/100 [00:00<?, ?it/s]

[Step 35000] loss: 0.0628
[Step 35010] loss: 0.0792
[Step 35020] loss: 0.0610
[Step 35030] loss: 0.0628
[Step 35040] loss: 0.0644
[Step 35050] loss: 0.0699
[Step 35060] loss: 0.0658
[Step 35070] loss: 0.0641
[Step 35080] loss: 0.0743
[Step 35090] loss: 0.0625
[Step 35100] loss: 0.0697
[Step 35110] loss: 0.0713
[Step 35120] loss: 0.0626
[Step 35130] loss: 0.0489
[Step 35140] loss: 0.0609
[Step 35150] loss: 0.0712
[Step 35160] loss: 0.0759
[Step 35170] loss: 0.0753
[Step 35180] loss: 0.0745
[Step 35190] loss: 0.0744
[Step 35200] loss: 0.0639
[Step 35210] loss: 0.0610
[Step 35220] loss: 0.0679
[Step 35230] loss: 0.0776
[Step 35240] loss: 0.0574
[Step 35250] loss: 0.0603
[Step 35260] loss: 0.0753
[Step 35270] loss: 0.0563
[Step 35280] loss: 0.0753
[Step 35290] loss: 0.0698
[Step 35300] loss: 0.0513
[Step 35310] loss: 0.0725
[Step 35320] loss: 0.0623
[Step 35330] loss: 0.0718
[Step 35340] loss: 0.0584
[Step 35350] loss: 0.0916
[Step 35360] loss: 0.0884
[Step 35370] loss: 0.0770
[Step 35380]

KeyboardInterrupt: 

## Inference

In [ ]:
ckpt = './UNetTransformer-emb256.pt'
trainer.load(ckpt)
# trainer.inference()

In [ ]:
diffusion = trainer.model
with torch.no_grad():
    all_steps = diffusion.sample(batch_size=1, return_all_timesteps=True)
# shape [1, T+1, C, H, W]

imgs = all_steps.squeeze(0).permute(0, 2, 3, 1).cpu().numpy()
imgs = unnormalize_to_zero_to_one(imgs)

timesteps = [0, diffusion.num_timesteps // 4, diffusion.num_timesteps // 2,
             3 * diffusion.num_timesteps // 4, diffusion.num_timesteps]
sel = [imgs[t] for t in timesteps]

# Plot
fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for ax, im, step in zip(axes, sel, timesteps):
    ax.imshow(im)
    ax.axis('off')
    ax.set_title(f"Step {step}")
plt.tight_layout()
plt.show()